# MLFlow Model Serving For Anaconda Enterprise
This notebook is meant to provide an exmaple of how to use the endpoint project to deploy endpoints using model from the MLFlow Tracking Server.

### 1. Create an instance of an AE Client

In [ ]:
from anaconda.enterprise.server.sdk.client import AEClient
from src.utils import get_ae_client

client: AEClient = get_ae_client()

### 2. Upload Endpoint Project

In [ ]:
from pathlib import Path
from anaconda.enterprise.server.contracts import ProjectUploadResponse

# Define the server template to upload
SERVER_TEMPLATE_PATH: str = Path("../../assets/mlflow.model.serving.0.4.0.tar.gz").resolve().as_posix()

# Define the project name
SERVER_PROJECT_NAME: str = "dev.mlflow.endpoint.taxi"

upload_response: ProjectUploadResponse = client.project_upload(
    project_archive_path=SERVER_TEMPLATE_PATH,
    name=SERVER_PROJECT_NAME,
)
upload_response

In [ ]:
project_revisions = client.project_revisions_get(project_id=upload_response.id)
project_revisions

#### 3. Define the Deployment Options
This will include specifying the registered model name and stage to load from the model registry.

In [ ]:
MLFLOW_SERVING_MODEL_NAME: str = "taxi_fare_regressor"
MLFLOW_SERVING_MODEL_STAGE: str = "Staging"
SERVER_DEPLOYMENT_NAME: str = "dev.mlflow.endpoint.taxi.fare.regressor"
SERVER_DEPLOYMENT_STATIC_ENDPOINT: str = "dev-mlflow-endpoint-taxi-fare-regressor"

deploy_params: dict = {
    "project_id": upload_response.id,
    "deployment_name": SERVER_DEPLOYMENT_NAME,
    "revision_id": project_revisions[0].id,
    "command": "Serve",
    "variables": {
        "MLFLOW_SERVING_MODEL_NAME": MLFLOW_SERVING_MODEL_NAME,
        "MLFLOW_SERVING_MODEL_STAGE": MLFLOW_SERVING_MODEL_STAGE,
    },
    "static_endpoint": SERVER_DEPLOYMENT_STATIC_ENDPOINT,
}
deploy_params

### 4. Deploy the Model Endpoint

In [ ]:
from anaconda.enterprise.server.contracts import ProjectDeployResponse

project_deploy_response: ProjectDeployResponse = client.project_deploy(**deploy_params)
project_deploy_response

### 5. Generate Private Access Token

In [ ]:
access_token: str = client.deployment_token_get(deployment_id=project_deploy_response.id)
access_token

### 6. Get Deployment URL

In [ ]:
service_endpoint: str = project_deploy_response.url
service_endpoint